In [ ]:
import sqlite3

class Product:
    def __init__(self, table_name):
        self.table_name = table_name
        self.conn = sqlite3.connect('C:RestauranManagementSystem.db')
        self.cursor = self.conn.cursor()
        self.create_table()
        
    def create_table(self):
        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {self.table_name}(
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                NAME VARCHAR(200),
                PRICE REAL NOT NULL
            )
        ''')
        self.conn.commit()
        
    def add_product(self, name, price):
        self.cursor.execute(f'''
            INSERT INTO {self.table_name}(NAME, PRICE) VALUES(?,?)
        ''', (name, price))
        self.conn.commit()
        print(f"{name} added.")
        
    def delete_product(self, product_id):
        self.cursor.execute(f"""
            DELETE FROM {self.table_name} WHERE ID=?
        """, (product_id,))
        self.conn.commit()
        print(f"{product_id} deleted")
        
    def update_product(self, product_id, name, price):
        self.cursor.execute(f"""
            UPDATE {self.table_name} SET NAME=?, PRICE=?
            WHERE ID=?
        """,(name, price, product_id))
        print(f"{product_id} has been updated.")
        
    def product_list(self):
        self.cursor.execute(f'SELECT ID, NAME, PRICE FROM {self.table_name}')
        products = self.cursor.fetchall()
        print(f"*********** {self.table_name.upper()} ***********")
        for product in products:
            print(f"ID: {product[0]}, Name: {product[1]}, Price: {product[2]} TL")
            
            
class Food(Product):
    def __init__(self):
        super().__init__('Foods')
        
class Beverage(Product):
    def __init__(self):
        super().__init__('Beverages')
        
class Orders:
    def __init__(self):
        self.conn = sqlite3.connect('C:RestauranManagementSystem.db')
        self.cursor = self.conn.cursor()
        self.create_table()
    
    def create_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS ORDERS(
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                FOOD_ID INTEGER NOT NULL,
                BEVERAGE_ID INTEGER NOT NULL,
                FOOD_COUNT INTEGER NOT NULL,
                BEVERAGE_COUNT INTEGER NOT NULL,
                TOTAL_PRICE REAL NOT NULL,
                FOREIGN KEY (FOOD_ID) REFERENCES FOODS(ID),
                FOREIGN KEY (BEVERAGE_ID) REFERENCES BEVERAGES(ID)
            )
        """)
        self.conn.commit()
        
    def take_order(self, food_id, beverage_id, food_count, beverage_count):
        food_price = self.get_product_price('Foods', food_id)
        beverage_price = self.get_product_price('Beverages', beverage_id)
        total_price = (food_price * food_count) + (beverage_count * beverage_price)
        self.cursor.execute("""
            INSERT INTO ORDERS(FOOD_ID, BEVERAGE_ID, FOOD_COUNT, BEVERAGE_COUNT, TOTAL_PRICE)
            VALUES(?,?,?,?,?)
        """, (food_id, beverage_id, food_count, beverage_count, total_price))
        self.conn.commit()
        print(f"{food_count} {self.get_product_name('Foods', food_id)} and {beverage_count} {self.get_product_name('Beverages', beverage_id)} has ordered . Total Price: {total_price}")
        
    def get_product_price(self, table, product_id):
        self.cursor.execute(f"SELECT PRICE FROM {table} WHERE ID=?", (product_id,))
        price = self.cursor.fetchone()
        if price:
            return price[0]
        else:
            print("Product not found.")
            return 0
        
    def get_product_name(self, table, product_id):
        self.cursor.execute(f"SELECT NAME FROM {table} WHERE ID=?", (product_id, ))
        name = self.cursor.fetchone()
        if name:
            return name[0]
        else:
            print("Product not found.")
            return 0
        
    def show_orders(self):
        self.cursor.execute('''
            SELECT S.ID, Y.NAME FOOD_NAME, I.NAME BEVERAGE_NAME, S.FOOD_COUNT FOOD_COUNT, 
            S.BEVERAGE_COUNT BEVERAGE_COUNT, S.TOTAL_PRICE TPRICE
            FROM ORDERS S
            INNER JOIN FOODS Y ON S.FOOD_ID = Y.ID
            INNER JOIN BEVERAGES I ON S.BEVERAGE_ID = I.ID
        ''')
        orders = self.cursor.fetchall()
        if not orders:
            print("There is no order yet.")
        else:
            print("Orders: ")
            for order in orders:
                print(f"""
                ID: {order[0]}, 
                Food: {order[1]}, Beverage: {order[2]}, 
                Food Count: {order[3]}, 
                Beverage Count: {order[4]}, 
                Total Price: {order[5]} TL
                """)
        
        
class RestaurantManagementSystem:
    def __init__(self):
        self.foods = Food()
        self.beverages = Beverage()
        self.orders = Orders()
        
    def take_order(self):
        self.foods.product_list()
        food_id = int(input("Food ID: "))
        food_count = int(input("Count: "))
        
        self.beverages.product_list()
        beverage_id = int(input("Beverage ID: "))
        beverage_count = int(input("Count: "))
        
        self.orders.take_order(food_id, beverage_id, food_count, beverage_count)
        
        
    def show_menu(self):
        while True:
            print(20*"*", "Restaurant Management System", 20*"*")
            print("""
                1. Add Food
                2. Add Beverage
                3. Delete Food
                4. Delete Beverage
                5. Update Food
                6. Update Beverage
                7. Show Foods
                8. Show Beverages
                9. Show Orders
                10. Take Order
                0. Quit
            """)
            secim = input("Please write a process: ")
            if secim == "1":
                self.foods.add_product(input("Food name: "), input("Food Price: "))
                
            elif secim == "2":
                self.beverages.add_product(input("Beverage name: "),input("Beverage price: "))
            
            elif secim == "3":
                self.foods.product_list()
                self.foods.delete_product(int(input("Food ID to Delete: ")))
                
            elif secim == "4":
                self.beverages.product_list()
                self.beverages.delete_product(int(input("Beverage ID to Delete: ")))
                
            elif secim == "5":
                self.foods.product_list()
                food_id = int(input("Food ID to Update: "))
                n_food_name = input("New Food Name: ")
                n_food_price = input("New Food Price: ")
                self.foods.update_product(food_id, n_food_name, n_food_price)
                
            elif secim == "6":
                self.beverages.product_list()
                b_id = int(input("Food ID to Update: "))
                n_b_name = input("New Food Name: ")
                n_b_price = input("New Food Price: ")
                self.beverages.update_product(b_id, n_b_name, n_b_price)
            elif secim == "7":
                self.foods.product_list()
                
            elif secim == "8":
                self.beverages.product_list()
                
            elif secim == "9":
                self.orders.show_orders()
                
            elif secim == "10":
                self.take_order()
                
            elif secim == "0":
                print("Turning off the system...")
                break
            else:
                print("Process not found!")
                
if __name__== "__main__":
    restaurant = RestaurantManagementSystem()
    restaurant.show_menu()


    

******************** Restaurant Management System ********************

                1. Add Food
                2. Add Beverage
                3. Delete Food
                4. Delete Beverage
                5. Update Food
                6. Update Beverage
                7. Show Foods
                8. Show Beverages
                9. Show Orders
                10. Take Order
                0. Quit
            
Please write a process: 1
Food name: Ayran
Food Price: 10
Ayran added.
******************** Restaurant Management System ********************

                1. Add Food
                2. Add Beverage
                3. Delete Food
                4. Delete Beverage
                5. Update Food
                6. Update Beverage
                7. Show Foods
                8. Show Beverages
                9. Show Orders
                10. Take Order
                0. Quit
            
Please write a process: 7
*********** FOODS ***********
ID: 1, Name: Ayran,